# LLM Optimization Modelling Experiment

In [3]:
import vertexai
from vertexai.preview.generative_models import GenerativeModel, Image
from IPython.display import Markdown

## 1. Define the problem description

In [55]:
problem = '''You are a chemist in charge of a process that requires two liquids to reach the end product. Your goal is to maximize the quality of the product in the end, which is measured by the difference between the Napierian logarithm of the product of the litres used of each component and the cube root of the summation of the first component litres as well as double the second component litres. It is required that the squared difference between the litres employed of each component and the mean litres utilised of both components is below ten. This is necessary for the obtained set to be stable. Furthermore, the mixture tank available to you can only fit 80 l. Please formulate a mathematical optimization model for this problem. '''

## 2. Generate the mathematical model

In [83]:
#Initializing the session. To replicate, make sure the right credentials are saved in a PATH variable
PROJECT_ID = "llm4optproblems"
REGION = "us-central1"
vertexai.init(project=PROJECT_ID, location=REGION)

#Specifying the model
generative_multimodal_model = GenerativeModel("gemini-1.5-pro-preview-0409")

#The propmt applied to all problems
prompt = '''Please formulate a mathematical optimization model for this problem. Include parameters, decision variables, the objective function and the constraints in your answer.
'''

#Generate the response
response = generative_multimodal_model.generate_content([prompt+problem])


In [84]:
#Show the resopnse in a formatted way
Markdown(response.text)

## Mathematical Optimization Model 

**Parameters:**

*  None specified in the problem.

**Decision Variables:**

*   $x_1$: Liters of the first liquid used.
*   $x_2$: Liters of the second liquid used.

**Objective Function:**

Maximize  $Z = \ln(x_1 \cdot x_2) - \sqrt[3]{x_1 + 2x_2}$

**Constraints:**

1. **Tank Capacity:** 
   $x_1 + x_2 \leq 80$

2. **Stability Constraint:**
   $(x_1 - \frac{x_1 + x_2}{2})^2 \leq 10$
   $(x_2 - \frac{x_1 + x_2}{2})^2 \leq 10$

3. **Non-negativity:**
   $x_1 \geq 0$
   $x_2 \geq 0$

**Summary:**

This model aims to find the optimal quantities of the two liquids ($x_1$, $x_2$) that maximize the product quality (Z) while ensuring the mixture fits within the tank capacity and maintains stability according to the specified criteria. 


## 3. Generate the pyomo code

In [85]:
#Second prompt gets the output of the previous step and generates the code
prompt2 = "Please write pyomo code for this mathematical problem. Use sample data where needed. Indicate where you use sample data. \n"
prompt2 += response.text 
response2 = generative_multimodal_model.generate_content([prompt2])

In [86]:
#Showing the code in a formatted way
Markdown(response2.text)

```python
import pyomo.environ as pyo

# Create a model
model = pyo.ConcreteModel()

# Define decision variables
model.x1 = pyo.Var(domain=pyo.NonNegativeReals)
model.x2 = pyo.Var(domain=pyo.NonNegativeReals)

# Define the objective function
model.obj = pyo.Objective(expr=pyo.log(model.x1 * model.x2) - (model.x1 + 2*model.x2)**(1/3), sense=pyo.maximize)

# Define the constraints
model.tank_capacity = pyo.Constraint(expr=model.x1 + model.x2 <= 80)
model.stability1 = pyo.Constraint(expr=(model.x1 - (model.x1 + model.x2)/2)**2 <= 10)
model.stability2 = pyo.Constraint(expr=(model.x2 - (model.x1 + model.x2)/2)**2 <= 10)

# Solve the model
solver = pyo.SolverFactory('ipopt') # You can choose a different solver if you prefer
results = solver.solve(model)

# Print the results
print("Solver status:", results.solver.status)
print("Termination condition:", results.solver.termination_condition)
print("Optimal solution:")
print("x1 =", pyo.value(model.x1))
print("x2 =", pyo.value(model.x2))
print("Objective function value (Z) =", pyo.value(model.obj))
```

This code defines the Pyomo model for the given mathematical optimization problem. No sample data was needed as all parameters are given within the problem statement. The code includes:

* **Defining decision variables:** `x1` and `x2` as non-negative real numbers.
* **Defining the objective function:** Maximizing  `Z = ln(x1*x2) - (x1 + 2*x2)**(1/3)`.
* **Defining the constraints:** Tank capacity, stability constraints, and non-negativity.
* **Solving the model:** Using the 'ipopt' solver (you can change this if needed).
* **Printing the results:** Solver status, termination condition, optimal values for `x1`, `x2`, and the objective function value. 


## 4. Input problem data and try running the generated code

In [87]:
import pyomo.environ as pyo

# Create a model
model = pyo.ConcreteModel()

# Define decision variables
model.x1 = pyo.Var(domain=pyo.NonNegativeReals)
model.x2 = pyo.Var(domain=pyo.NonNegativeReals)

# Define the objective function
model.obj = pyo.Objective(expr=pyo.log(model.x1 * model.x2) - (model.x1 + 2*model.x2)**(1/3), sense=pyo.maximize)

# Define the constraints
model.tank_capacity = pyo.Constraint(expr=model.x1 + model.x2 <= 80)
model.stability1 = pyo.Constraint(expr=(model.x1 - (model.x1 + model.x2)/2)**2 <= 10)
model.stability2 = pyo.Constraint(expr=(model.x2 - (model.x1 + model.x2)/2)**2 <= 10)

# Solve the model
solver = pyo.SolverFactory('ipopt') # You can choose a different solver if you prefer
results = solver.solve(model)

# Print the results
print("Solver status:", results.solver.status)
print("Termination condition:", results.solver.termination_condition)
print("Optimal solution:")
print("x1 =", pyo.value(model.x1))
print("x2 =", pyo.value(model.x2))
print("Objective function value (Z) =", pyo.value(model.obj))

Solver status: ok
Termination condition: optimal
Optimal solution:
x1 = 43.162277233647174
x2 = 36.8377229631074
Objective function value (Z) = 2.4827783318454744


## 5. Correct the code to verify model viability (optional)

## 6. Printing the outputs as strings, so they can be saved.
Those can be rendered as markdown for better readability

In [88]:
print(response.text)

## Mathematical Optimization Model 

**Parameters:**

*  None specified in the problem.

**Decision Variables:**

*   $x_1$: Liters of the first liquid used.
*   $x_2$: Liters of the second liquid used.

**Objective Function:**

Maximize  $Z = \ln(x_1 \cdot x_2) - \sqrt[3]{x_1 + 2x_2}$

**Constraints:**

1. **Tank Capacity:** 
   $x_1 + x_2 \leq 80$

2. **Stability Constraint:**
   $(x_1 - \frac{x_1 + x_2}{2})^2 \leq 10$
   $(x_2 - \frac{x_1 + x_2}{2})^2 \leq 10$

3. **Non-negativity:**
   $x_1 \geq 0$
   $x_2 \geq 0$

**Summary:**

This model aims to find the optimal quantities of the two liquids ($x_1$, $x_2$) that maximize the product quality (Z) while ensuring the mixture fits within the tank capacity and maintains stability according to the specified criteria. 



In [89]:
print(response2.text)

```python
import pyomo.environ as pyo

# Create a model
model = pyo.ConcreteModel()

# Define decision variables
model.x1 = pyo.Var(domain=pyo.NonNegativeReals)
model.x2 = pyo.Var(domain=pyo.NonNegativeReals)

# Define the objective function
model.obj = pyo.Objective(expr=pyo.log(model.x1 * model.x2) - (model.x1 + 2*model.x2)**(1/3), sense=pyo.maximize)

# Define the constraints
model.tank_capacity = pyo.Constraint(expr=model.x1 + model.x2 <= 80)
model.stability1 = pyo.Constraint(expr=(model.x1 - (model.x1 + model.x2)/2)**2 <= 10)
model.stability2 = pyo.Constraint(expr=(model.x2 - (model.x1 + model.x2)/2)**2 <= 10)

# Solve the model
solver = pyo.SolverFactory('ipopt') # You can choose a different solver if you prefer
results = solver.solve(model)

# Print the results
print("Solver status:", results.solver.status)
print("Termination condition:", results.solver.termination_condition)
print("Optimal solution:")
print("x1 =", pyo.value(model.x1))
print("x2 =", pyo.value(model.x2))
print("